In [1]:
from eth_account import Account
from x402.clients.requests import x402_requests
from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware, LocalFilterMiddleware
import json
import time
import os
from dotenv import load_dotenv
import pandas as pd
from chartengineer import ChartMaker
from typing import Optional
import sys

load_dotenv()

True

In [2]:
RPC_URL = os.getenv("RPC_URL")
EXPLORER_URL = os.getenv("EXPLORER_URL")
USDC_ADDRESS_RAW = os.getenv("USDC_ADDRESS", "0xB97EF9Ef8734C71904D8002F8b6Bc66Dd9c48a6E")
USDC_ADDRESS = Web3.to_checksum_address(USDC_ADDRESS_RAW)

print(f'Using RPC URL: {RPC_URL} and Explorer URL: {EXPLORER_URL}')

Using RPC URL: https://avax-mainnet.g.alchemy.com/v2/RGdonbHyjNzV5McbhRpfy and Explorer URL: https://snowscan.xyz/tx/


In [3]:
w3 = Web3(Web3.HTTPProvider(RPC_URL))
w3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)
w3.middleware_onion.add(LocalFilterMiddleware)

In [4]:
BASE_DIR = os.getcwd()

print(f"BASE_DIR: {BASE_DIR}")

BASE_DIR: c:\Users\brand\projects\galaksio\glusd\analytics\notebooks


In [5]:
GLUSD_ABI_PATH = os.path.join(BASE_DIR, "..","..", "contracts", "out", "GLUSD.sol", "GLUSD.json")
with open(GLUSD_ABI_PATH, 'r') as f:
    GLUSD_ABI = json.load(f)["abi"]

ERC20_ABI_PATH = os.path.join(BASE_DIR, "..","..", "contracts", "out", "ERC20.sol", "ERC20.json")
with open(ERC20_ABI_PATH, 'r') as f:
    ERC20_ABI = json.load(f)["abi"]

GLUSD_ADDRESS_PATH = os.path.join(BASE_DIR, "..", "..", "contracts", "deployments", "GLUSD.json")
with open(GLUSD_ADDRESS_PATH, 'r') as f:
    GLUSD_ADDRESS = w3.to_checksum_address(json.load(f)['deployedTo'])

In [6]:
glusd_contract = w3.eth.contract(address=GLUSD_ADDRESS, abi=GLUSD_ABI)

usdc_contract = w3.eth.contract(
    address=USDC_ADDRESS,
    abi=ERC20_ABI
)

glusd_decimals = glusd_contract.functions.decimals().call()
usdc_decimals = usdc_contract.functions.decimals().call()

In [7]:
# GLUSD Supply, USDC Backing

vault_usdc_balance, glusd_supply = glusd_contract.functions.vaultStatus().call()
print(f"GLUSD Vault USDC Balance: {vault_usdc_balance / 10 ** usdc_decimals}")
print(f"GLUSD Supply from Vault Status: {glusd_supply / 10 ** glusd_decimals}")

GLUSD Vault USDC Balance: 6.278086
GLUSD Supply from Vault Status: 6.055057


In [8]:
# Max Mint Limit

MAX_TOTAL_SUPPLY = glusd_contract.functions.MAX_TOTAL_SUPPLY().call()
print(f"GLUSD Max Total Supply: {MAX_TOTAL_SUPPLY / 10 ** glusd_decimals}")

GLUSD Max Total Supply: 1000.0


In [9]:
# % Minted of Max Supply, Amount Left to Mint

percent_minted = (glusd_supply / MAX_TOTAL_SUPPLY) * 100
amount_left_to_mint = MAX_TOTAL_SUPPLY - glusd_supply


print(f"GLUSD Percent Minted of Max Supply: {percent_minted}%")
print(f"GLUSD Amount Left to Mint: {amount_left_to_mint / 10 ** glusd_decimals}")

GLUSD Percent Minted of Max Supply: 0.6055057%
GLUSD Amount Left to Mint: 993.944943


In [10]:
# Exchange Rate (GLUSD to USDC)

exchange_rate = glusd_contract.functions.exchangeRate().call()
print(f"GLUSD/USDC Exchange Rate: {exchange_rate / 10 ** glusd_decimals}")

GLUSD/USDC Exchange Rate: 1.036833


In [11]:
# Last Snapshot and Snapshot Interval

last_snapshot_time = glusd_contract.functions.lastSnapshotTime().call()
print(f"GLUSD Last Snapshot Time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(last_snapshot_time))}")

min_snapshot_interval = glusd_contract.functions.MIN_SNAPSHOT_INTERVAL().call()
print(f"GLUSD Min Snapshot Interval: {min_snapshot_interval} seconds")

GLUSD Last Snapshot Time: 2025-12-12 11:32:18
GLUSD Min Snapshot Interval: 30 seconds


In [12]:
# BP Scale

bp_scale = glusd_contract.functions.BP_SCALE().call()
print(f"GLUSD BP Scale: {bp_scale}")

GLUSD BP Scale: 10000


In [13]:
apr_7_d, apr_30_d = glusd_contract.functions.getCurrentAPRs().call()

apr7 = apr_7_d / 1e6   # convert from 5_000_000 → 5.0 (%)
apr30 = apr_30_d / 1e6

print(f"GLUSD Current APRs: 7-day APR: {apr7:.2f}%, 30-day APR: {apr30:.2f}%")


GLUSD Current APRs: 7-day APR: 14470.73%, 30-day APR: 14470.73%


In [14]:
# X-Day APR and APY

days = [1, 7, 30, 90]

for day in days:
    apr = glusd_contract.functions.calculateAPR(day).call()
    print(f"{day}-Day GLUSD APR: {apr / 100}%")

    apy = glusd_contract.functions.calculateAPY(day).call()
    print(f"{day}-Day GLUSD APY: {apy / 100}%")

1-Day GLUSD APR: 144707298.86%
1-Day GLUSD APY: 144680024.0%
7-Day GLUSD APR: 144707298.86%
7-Day GLUSD APY: 144680024.0%
30-Day GLUSD APR: 144689273.54%
30-Day GLUSD APY: 144680024.0%
90-Day GLUSD APR: 144689273.54%
90-Day GLUSD APY: 144680024.0%


In [15]:
# Historical Exchange Rate Over Time

In [16]:
# Historical APR and APY Over Time